# Direct and inverse geometry of 3d robots
In the previous class, we worked with 2d robot. Let's now move to 3D cases, with a real manipulator robot. To begin, let's discuss quickly 3D rotations and placements.
## Rotations and placement, SO(3) and SE(3)
A placement is composed of a rotation and a translation. They are represented in Pinocchio as SE3 objects, containing a 3x3 unit matrix and a 3x1 vector.

In [ ]:
import pinocchio
from pinocchio import SE3,Quaternion
import numpy as np
from numpy.linalg import norm
M = SE3.Random()
R = M.rotation
p = M.translation
print('R =\n'+str(R))
print('p =\n'+str(p.T))

A rotation is simply a 3x3 matrix. It has a unit norm:

In [ ]:
print(R*R.T)

It can be equivalently represented by a quaternion. Here we have made the choice to create a specific class for quaternions (i.e. they are not vectors, and can be e.g. multiplied), but you can get the 4 coefficients with the adequate method. Note that the corresponding vector is also of norm 1.

In [ ]:
quat = Quaternion(R)
print(norm(quat.coeffs()))

Angle-axis representation are also implemented in the class AngleAxis. In case of errors at AngleAxis importation, just modify your file /opt/openrobots/lib/python2.7/site-packages/pinocchio/__init__.py to comment the last line (sorry for the bug).

In [ ]:
from pinocchio import AngleAxis
utheta = AngleAxis(quat)
print(utheta.angle, utheta.axis.T)

You can display rotation in Gepetto Viewer (remember to first run gepetto-gui from command line to start the viewer).

In [ ]:
import gviewserver
gv = gviewserver.GepettoViewerServer()
gv.addBox ('world/box',    .1,.2,.3,    [1 ,0 ,0,1])
gv.applyConfiguration('world/box',[.1,.2,.3]+quat.coeffs().T.tolist()[0])
gv.refresh()

### Quaternion you said?
Quaternions are "complex of complex", introduced form complex as complex are from reals. Let's try to understand what they contains in practice. Quaternions are of the form w+xi+yj+zk, with w,x,y,z real values, and i,j,k the 3 imaginary numbers. We store them as 4-d vectors, with the real part first: quat = [x,y,z,w]. We can interprete w as encoding the cosinus of the rotation angle. Let's see that.

In [ ]:
from numpy import arccos
print(arccos(quat[3]))
print(AngleAxis(quat).angle)

Indeed, w = cos(theta/2). Why divided by two? For that, let's see how the quaternion can be used to represent a rotation. We can encode a 3D vector in the imaginary part of a quaternion.

In [ ]:
from pinocchio.utils import rand
p = rand(3)
qp = Quaternion(0.,p[0,0],p[1,0],p[2,0])
print(qp)

The real product extends over quaternions, so let's try to multiply quat with p:

In [ ]:
print(quat*qp)

Well that's not a pure imaginary quaternion anymore. And the imaginary part does not contains somethig that looks like the rotated point:

In [ ]:
print((quat.matrix()*p).T)

The pure quaternion is obtained by multiplying again on the left by the conjugate (w,-x,-y,-z).

In [ ]:
print(quat*qp*quat.conjugate())

That is a pure quaternion, hence encoding a point, and does corresponds to R*p. Magic, is it not? We can prove that the double product of quaternion does corresponds to the rotation. Indeed, a quaternion rather encode an action (a rotation) in $R^4$, but which moves our point p outside of $R^3$. The conjugate rotation brings it back in $R^3$ but applies a second rotation. Since we rotate twice, it is necessary to apply only half of the angle each time.
What if we try to apply the rotation quat on the imaginary part of the quaternion?

In [ ]:
qim = Quaternion(quat) # copy
qim[3] = 0
print(qim, quat*qim*quat.conjugate())

What kind of conclusion can we get from this? What geometrical interpretation can we give to $q_{im}$? What about $||q_{im}||$?

### The SLERP example
Let's practice! Implement a linear interpolation between two position p0 and p1, i.e. find the position p(t) with t varying from 0 to 1, with p(0)=p0, p(1)=p1 and continuity between the two extrema.

In [ ]:
# %load tp2/solution_lerp.py

LERP with quaternions is not working because they are not normalize. Instead we can take either the normalization of the LERP (NLERP), or the spherical LERP (SLERP). 

In [ ]:
# %load tp2/solution_slerp.py

## 3D robots
Let's now play with 3D robots. We will load the models from URDF files.

*The robot UR5* is a low-cost manipulator robot with good performances. It is a fixed robot with one 6-DOF arms developed by the Danish company Universal Robot. All its 6 joints are revolute joints. Its configuration is in R^6 and is not subject to any constraint. The model of UR5 is described in a URDF file, with the visuals of the bodies of the robot being described as meshed (i.e. polygon soups) using the Collada format ".dae". Both the URDF and the DAE files are available in the repository in the model directory. 

*The robot Romeo* is a humanoid robot, with 4 limbs (each with 6 joints, a torso and a neck. We will use it as an example of mobile robot, whose first joint is a "free floating" virtual link.


In [1]:
# %load tp2/example_robots.py
WITH_UR5   = True
WITH_ROMEO = False

def loadUr5():
    from pinocchio.robot_wrapper import RobotWrapper

    path  = 'models/'
    urdf = path + '/ur_description/urdf/ur5_gripper.urdf'
    return RobotWrapper(urdf,[path,])

def loadRomeo():
    from pinocchio.romeo_wrapper import RomeoWrapper

    path = 'models/romeo/'
    urdf = path + 'urdf/romeo.urdf'

    # Explicitly specify that the first joint is a free flyer.
    return RomeoWrapper(urdf,[path,]) # Load urdf model

if WITH_UR5: robot = loadUr5()
else: assert(WITH_ROMEO); robot = loadRomeo()

robot.initDisplay(loadModel=True)
robot.display(robot.q0)

The kinematic tree is represented by two C++ objects called Model (which contains the model constants: lengths, masses, names, etc) and Data (which contains the working memory used by the model algorithms). Both C\++ objects are contained in a unique Python class. The first class is called RobotWrapper and is generic.

For the next steps, we are going to work with the RobotWrapper.

Import the class RobotWrapper and create an instance of this class in the python terminal. At initialization, RobotWrapper will read the model description in the URDF file given as argument. In the following, we will use the model of the UR5 robot, available in the directory "models" of pinocchio (available in the homedir of the VBox). The code of the RobotWrapper class is in /opt/openrobots/lib/python2.7/site-packages/pinocchio/robot_wrapper.py . Do not hesitate to have a look at it and to take inspiration from the implementation of the class functions.

Here are some import methods of the class.
* robot.index('joint name') returns the index of the joint.
* robot.model.names is a container (~list) that contains all the joint names
* robot.model.frames contains all the import frames attached to the robot. 
* robot.placement(idx) and robot.framePlacement(idx) returns the placement (i.e. translation+rotation of the joint / frame in argument.
* robot.display(q) display the configuration q.
* robot.q0 contains a basic configuration.

### Simple pick and place

Say we have a target at position [.5,.1,.2] and we would like the robot to grasp it.

In [2]:
gv = robot.viewer.gui
gv.addSphere("world/sphere", .1, [1,0,0,1])
gv.applyConfiguration("world/sphere",[.5,.1,.2, 1.,0.,0.,0.])
gv.refresh() 

Then decide by any mean you want a configuration of the robot so that the end effector is touching the sphere.

At the reference position you built, the end effector placement can be obtained by robot.position(q,6). Only the translation part of the placement has been selected. The rotation is free.

Say now that the object is a rectangle and not a sphere. Pick the object at a reference position with the rotation that is imposed, so that the end effector is aligned with one of the faces of the rectangle.

Now, choose any trajectory you want in the configuration space, starting from the reference position built in the previous exercice (it can be sinus-cosinus waves, polynomials, splines, straight lines).

Make a for loop to display the robot at sampling positions along this trajectory. The function sleep in module time (from time import sleep) can be used to slow down the loop.

At each instant of your loop, recompute the position of the ball and display it so that it always "sticks" to the robot end effector.


In [3]:
# % load tp2/simple_pick_and_place.py

### Inverse geometry in 3D
Let's now first control the position (i.e. translation only) of the end effector of a manipulator robot to a given position. For this first part, we will use the fixed serial-chain robot model.

Recall first that the position (3D) of the joint with index "i" at position "q" can be access by the following two lines of code.

In [4]:
q = rand(robot.model.nq)
idx = robot.index( 'wrist_3_joint' )
pinocchio.forwardKinematics(robot.model,robot.data,q)
p = robot.data.oMi[idx].translation

NameError: name 'rand' is not defined

Using the scipy solver used in the 2D geometry notebook, compute a configuration q where the end effector reaches p.

In [ ]:
# %load tp2/solution_invgeom3d.py

### Inverse geometry in 6D
6D means: translation and rotation. Change the previous cost function for a cost measuring the difference between the current placement root.placement(q,6) and a reference placement oMdes. 

In [ ]:
# %load tp2/solution_invgeom6d.py

We can check in the last computed placement that we reached the target:

In [ ]:
print robot.data.oMi[6]

### Optimizing in the quaternion space

Let's now work with the robot Romeo. This robot has 31 joints, but Q-space of size 38 (robot.model.nq) and Q-tangent space of size 37 (robot.model.nv). Because with need 7D vector to encode the robot placement in space, which indeed to only 6 DOF.

First modify the UR5 inverse-geometry code to control the robot feet and hands position. Before runing the code, try to imagine what the solve will do with your quaternion.

In [ ]:
# %load tp2/solution_invgeom_quat.py
from numpy.linalg import norm

robot = loadRomeo()
robot.initDisplay(loadModel=True)

target_rh  = np.matrix([1.5, 0.1, 1.5 ]).T  # x,y,z
target_rf  = np.matrix([0., -0.1, 0.0 ]).T  # x,y,z
target_lf  = np.matrix([0.,  0.1, 0.0 ]).T  # x,y,z
target_com = np.matrix([0.0, 0.0, 0.5 ]).T  # x,y,z

def cost(q):
    q = np.matrix(q).T
    error  = 0.0
    error += norm(robot.placement(q, robot.rh).translation - target_rh )**2
    error += norm(robot.placement(q, robot.rf).translation - target_rf )**2 * 100
    error += norm(robot.placement(q, robot.lf).translation - target_lf )**2 * 100
    error += norm(robot.com(q)                             - target_com)**2 * 10
    return error

def callback(q):
    q = np.matrix(q).T
    robot.display(q)
    time.sleep(.1)

robot.display(robot.q0)
qopt = fmin_bfgs(cost, robot.q0, callback=callback)


Modify the cost so that the quaternion is kept around one.

## Homework
A parallel robot is composed of several kinematic chains (called the robot legs) that are all attached to the same end effector. This imposes strict constraints in the configuration space of the robot: a configuration is valide iff all the legs meets the same end-effector placement. We consider here only the geometry aspect of parallel robots (additionnally, some joints are not actuated, which causes additional problems).

The kinematic structure of a paralel robot indeed induces loops in the joint connection graph. In Pinocchio, we can only represents (one of) the underlying kinematic tree. The loop constraints have to be handled separately. An example that loads 4 manipulator arms is given below.

In [ ]:
% load tp2/example_parallel.py

 Each leg i (for i=0,1,2,3) of the robot is loaded in the list robots[i]. The loop constraints are that the relative placement of every leg end-effector must stay the same that in the initial configuration given as example in the above file.

Consider now that the orientation of the tool plate is given by the quaternion Quaternion(0.7,0.2,0.2,0.6), with the translation that you like. Find using the above optimization routines the configuration of each robot leg so that the loop constraints are all met.

You can convert quaternion to rotation matrix and create SE3 objects as follows:


In [ ]:
qu = Quaternion(.7,.2,.2,.6)# Quaternion: take care that norm <= 1 (and approx 1)
R  = qu.matrix()                # Create a rotation matrix from quaternion
p  = np.matrix([0.,0.,0.77]).T  # Translation (R3) vector)
M  = SE3(R,p)                   # Create a nomogeneous matrix from R,P

# Typical tool position
from pinocchio.utils import rotate
M = SE3(rotate('z',1.)*rotate('x',.2), np.matrix([0.1,0.02,.65]).T)


### Instructions 

Send by mail at nmansard@laas.fr (NOT the supaero address, I don't read it) a mail containing a single python file. The subject of the mail should start with [SUPAERO] TP2. When executed, the script should execute the placement of the quadripod robot.
